In [1]:
import numpy as np
import requests
from PIL import Image

In [2]:
def get_from_url(url):
    r = requests.get(url)

    with open("/tmp/image.jpg", 'wb') as f:
        f.write(r.content) 


def preprocess(X):
    return np.float32((X/127.5) - 1)


In [3]:
def get_url(url):
    get_from_url(url)

    with Image.open('/tmp/image.jpg') as img:
        img = img.resize((299, 299), Image.NEAREST)

    X = preprocess(np.array([np.array(img)]))
    
    return X

In [4]:
X = get_url('https://bit.ly/3N3qWuF')

In [5]:
#! pip install tensorflow-serving-api

In [6]:
import grpc
import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [7]:
X_proto = tf.make_tensor_proto(X, shape=X.shape)

In [8]:
host = 'localhost:8500'
channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [9]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_2'].CopyFrom(X_proto)

In [10]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [11]:
prediction = pb_response.outputs['dense_1'].float_val

In [12]:
classes = ['dress','hat','longsleeve','outwear','pants','shirt','shoes','shorts','skirt','t-shirt']

In [14]:
result = dict(zip(classes, prediction))

In [15]:
result

{'dress': 2.4360194206237793,
 'hat': -6.271371364593506,
 'longsleeve': 2.911688804626465,
 'outwear': -5.460511207580566,
 'pants': -8.456361770629883,
 'shirt': 3.5294289588928223,
 'shoes': -10.453047752380371,
 'shorts': -4.042233467102051,
 'skirt': -5.232727527618408,
 't-shirt': 7.850218772888184}